In [23]:
import pandas as pd
import numpy as np

In [24]:
def gcd(a,b):
    a = abs(a)
    b = abs(b)
    if min(a,b)==0:
        return max(a,b)
    while b% a !=0:
        r = b% a
        b = a
        a = r
    return a

def gcd_list(l):
    d = 0
    for a in l:
        d = gcd(a,d)
    return d

def removecommonfactor(abc):
    a,b,c = abc
    d = gcd_list(abc)
    if d == 0:
        return abc
    return a//d,b//d,c//d

def translaterootbqf(abc,n):
    a,b,c = abc
    return (a,-2*a*n+b,a*n*n-b*n+c)

def rescalenum(abc,n):
    a,b,c = abc
    return (a,n*b,n*n*c)

def rescaleden(abc,n):
    a,b,c = abc
    return (n*n*a,n*b,c)
    
def primefac(n):
    if n == 0:
        return {0:1}
    pf = {}
    if n < 0:
        pf[-1]=1
        n = abs(n)
    if n % 2 == 0:
        pf[2] = 0
        while n % 2 == 0:
            n = n//2
            pf[2]+=1
    if n % 3 == 0:
        pf[3] = 0
        while n % 3 == 0:
            n = n//3
            pf[3]+=1
    d = 5
    e = -1
    while d*d <= n:
        if n % d == 0:
            pf[d] = 0
            while n % d == 0:
                n = n//d
                pf[d]+=1
        d+=3+e
        e*=-1
    if n > 1:
        pf[n]=1
    return pf

def pf_to_int(pf):
    n = 1
    for p in pf:
        n*= p**pf[p]
    return n

def quad_gcd(a,b):
    pf1 = primefac(a)
    pf2 = primefac(b)
    if a == 0:
        pfgcd = {p:pf2[p]//2 for p in pf2}
        return pf_to_int(pfgcd)
    pfgcd = {}
    for p in pf1:
        if p in pf2:
            pfgcd[p] = min(pf1[p],pf2[p]//2)
    return pf_to_int(pfgcd)

def discfac(d):
    pfd = primefac(d)
    rt = pf_to_int({p:pfd[p]//2 for p in pfd})
    fd = d//(rt*rt)
    if fd % 4 > 1:
        fd*=4
        rt = rt//2
    return fd, rt

id2 = np.matrix([[1,0],[0,1]])
def matrixgenfromd(d):
    return np.matrix([[0,d//4],[1,-(d%4)]])

def frobmatrixfromminpoly(ap):
    a,p = ap
    d = a*a-4*p
    d0,m = discfac(d)
    one = np.matrix([[1,0],[0,1]])
    tau = matrixgenfromd(d0)
    if d0 % 4 == 0:
        return (a//2)*one+m*tau
    else:
        return ((a+m)//2) *one + m*tau
    
def gcdmat(mat):
    arr = np.array(mat).reshape(1,-1)[0]
    return gcd_list([int(a) for a in arr])

def matdet(m):
    m = np.array(m)
    return m[0,0]*m[1,1]-m[0,1]*m[1,0]

def mattr(m):
    m = np.array(m)
    return m[0,0]+m[1,1]

def mwexp_frommat(m):
    t = mattr(m)
    d = matdet(m)
    return abs(quad_gcd(t,d))

def matconj(m):
    return m - mattr(m)*np.matrix([[1,0],[0,1]])
    
def mordell_weil_gens(ap,n):
    frob = frobmatrixfromminpoly(ap)
    a,p = ap
    d = a*a-4*p
    d0,m = discfac(d)
    one = np.matrix([[1,0],[0,1]])
    tau = matrixgenfromd(d0)
    frobpowerminusone= np.linalg.matrix_power(frob,n)-np.matrix([[1,0],[0,1]])
    mw2exp = mwexp_frommat(frobpowerminusone)
    mwcycmat = frobpowerminusone//mw2exp
    cycorder = matdet(mwcycmat)
    if cycorder == 1:
        return ((0,1),mw2exp,mw2exp)
    biggenorder = mw2exp*cycorder
    generator = np.array(matconj(frobpowerminusone))
    x = generator[0,0]%biggenorder
    y = generator[1,0]%biggenorder
    xygcd = gcd(x,y)
    gen1 = (x//xygcd,y//xygcd)
    return gen1,mw2exp,biggenorder

def cx_to_arr(z)->np.array:
    return np.array([z.real,z.imag])

def pointgroup(ap,n):
    frob = frobmatrixfromminpoly(ap)
    a,p = ap
    d = a*a-4*p
    d0,m = discfac(d)
    frobpowerminusone= np.linalg.matrix_power(frob,n)-np.matrix([[1,0],[0,1]])
    mw2exp = mwexp_frommat(frobpowerminusone)
    mwcycmat = frobpowerminusone//mw2exp
    cycorder = matdet(mwcycmat)
    if cycorder == 1:
        one_arr = np.array([1,0])
        taumat = matrixgenfromd(d0)
        tau_cp_coefs = [1,-mattr(taumat),matdet(taumat)]
        tau_cx = np.roots(tau_cp_coefs)[0]
        tau_arr = cx_to_arr(tau_cx)
        return np.array([a*one_arr+b*tau_arr
                         for a in range(mw2exp)
                         for b in range(mw2exp)])/mw2exp
    biggenorder = mw2exp*cycorder
    generator = np.array(matconj(frobpowerminusone))
    x = generator[0,0]%biggenorder
    y = generator[1,0]%biggenorder
    xygcd = gcd(x,y)
    xg,yg = (x//xygcd,y/xygcd)
    gen1intarr = np.array([xg,yg])

    #Need this to make sure we pick the second generator
    #outside of the subgroup generated by first generator
    cycgroupints = {((s*xg)%biggenorder,(s*yg)%biggenorder) 
                    for s in range(1,biggenorder)}
    if min([xy[0] for xy in cycgroupints if xy!=(0,0)]) == 0:
        gen2intarr = np.array([0,cycorder])
    else:
        gen2intarr = np.array([cycorder,0])
    xyints_all = [(a*gen1intarr+b*gen2intarr)%biggenorder 
                  for a in range(biggenorder)
                  for b in range(mw2exp)]
    one_arr = np.array([1,0])
    taumat = matrixgenfromd(d0)
    tau_cp_coefs = [1,-mattr(taumat),matdet(taumat)]
    tau_cx = np.roots(tau_cp_coefs)[0]
    tau_arr = cx_to_arr(tau_cx)
    points = np.array([xy[0]*one_arr+xy[1]*tau_arr for xy in xyints_all])/biggenorder
    return points

In [21]:
jinv_minimal_polynomials = pd.read_pickle('../data/dataframes/hilb_lifts.pk')
eccountdata = pd.read_pickle('../data/dataframes/elliptic_curve_counts.pk')

In [25]:
eccountdata.loc[eccountdata.Discriminant==-8]

,TraceFrob,Prime,Discriminant,DiscriminantFac,jInvariants,degree
ap,,,,,,
"(6, 11)",6,11,-8,"(-8, 1)",[3],1
"(18, 83)",18,83,-8,"(-8, 1)",[32],1
"(30, 227)",30,227,-8,"(-8, 1)",[55],1
"(42, 443)",42,443,-8,"(-8, 1)",[26],1
"(66, 1091)",66,1091,-8,"(-8, 1)",[363],1
"(78, 1523)",78,1523,-8,"(-8, 1)",[385],1
"(90, 2027)",90,2027,-8,"(-8, 1)",[1919],1
"(114, 3251)",114,3251,-8,"(-8, 1)",[1498],1
"(162, 6563)",162,6563,-8,"(-8, 1)",[1437],1


In [14]:
hilblifts

,coefs,ReducedDisc,Conductor,deg
d,,,,
-3,"[0, 1]",-3,1,1
-4,"[-1728, 1]",-4,1,1
-8,"[-8000, 1]",-8,1,1
-11,"[32768, 1]",-11,1,1
-12,"[0, -54000, 1]",-3,2,2
...,...,...,...,...
-30067,[-93541070534412252588715194722936575722175679...,-30067,1,14
-32323,[638401687605604061390314876094254404368703460...,-32323,1,15
-34483,[148403539327448096253508622631783924020320678...,-34483,1,15


In [19]:
eccountdata['degree'] = eccountdata.degree *(1+(eccountdata.TraceFrob==0))

In [20]:
eccountdata[['TraceFrob','Prime', 'Discriminant', 'DiscriminantFac', 'jInvariants',
       'degree']].to_pickle('../data/dataframes/elliptic_curve_counts.pk')